# Block 6 Exercise 1: Non-Linear Classification

## MNIST Data
We return to the MNIST data set on handwritten digits to compare non-linear classification algorithms ...   

In [1]:
#imports 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml

In [2]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)


In [3]:
#the full MNIST data set contains 70k samples of digits 0-9 as 28*28 gray scale images (represented as 784 dim vectors)
np.shape(X)

(70000, 784)

In [4]:
X.min()

0.0

In [5]:
#look at max/min value in the data
X.max()

255.0

### E1.1: Cross-Validation and Support Vector Machines
Train and optimize  C-SVM classifier on MNIST (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
* use a RBF kernel
* use *random search* with cross-validation to find the best settings for *gamma* and *C* (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)
* use max_iter in the SVM to avoid long training times 

In [6]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform,loguniform

svm = SVC(gamma='auto',kernel='rbf',max_iter=100)

distributions = dict(C=uniform(2,4),
                      gamma=uniform(0.8, 1))
clf = RandomizedSearchCV(svm, distributions, random_state=0,n_jobs=6)
search = clf.fit(X, y)

G:\Users\David\anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [7]:
search.best_params_

{'C': 4.195254015709299, 'gamma': 1.5151893663724194}

### E1.2: Pipelines and simple Neural Networks
Split the MNIST data into  train- and test-sets and then train and evaluate a simple Multi Layer Perceptron (MLP) network. Since the non-linear activation functions of MLPs are sensitive to the scaling on the input (recall the *sigmoid* function), we need to scale all input values to [0,1] 

* combine all steps of your training in a SKL pipeline (https://scikit-learn.org/stable/modules/compose.html#pipeline)
* use a SKL-scaler to scale the data (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
* MLP Parameters: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier
    * use a *SGD* solver
    * use *tanh* as activation function
    * compare networks with 1, 2 and 3 layers, use different numbers of neurons per layer
    * adjust training parameters *alpha* (regularization) and *learning rate* - how sensitive is the model to these parameters?
    * Hint: do not change all parameters at the same time, split into several experiments
* How hard is it to find the best parameters? How many experiments would you need to find the best parameters?
    


In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score

from scipy.stats import randint

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [7]:
MLP=make_pipeline(StandardScaler(), 
                       MLPClassifier(solver="sgd",activation="tanh",max_iter=300)
                      )

parameter_dist_1 = {
    'mlpclassifier__hidden_layer_sizes': [
        (250,75,150),                                          
        (125,100,),
        (75,)
    ]    
}

clf_hidden_layer = GridSearchCV(MLP, parameter_dist_1,n_jobs=6)
search_hidden_layer = clf_hidden_layer.fit(X_train,y_train)

In [8]:
search_hidden_layer.cv_results_

{'mean_fit_time': array([1197.62055106,  779.17209044,  457.96134601]),
 'std_fit_time': array([ 8.24785978, 29.46181064, 64.01808981]),
 'mean_score_time': array([0.59640541, 0.31854677, 0.19807773]),
 'std_score_time': array([0.03905095, 0.06067985, 0.05602251]),
 'param_mlpclassifier__hidden_layer_sizes': masked_array(data=[(250, 75, 150), (125, 100), (75,)],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'mlpclassifier__hidden_layer_sizes': (250, 75, 150)},
  {'mlpclassifier__hidden_layer_sizes': (125, 100)},
  {'mlpclassifier__hidden_layer_sizes': (75,)}],
 'split0_test_score': array([0.96063492, 0.96222222, 0.95571429]),
 'split1_test_score': array([0.95992063, 0.95857143, 0.96007937]),
 'split2_test_score': array([0.96309524, 0.96071429, 0.95968254]),
 'split3_test_score': array([0.95873016, 0.96126984, 0.95666667]),
 'split4_test_score': array([0.96253968, 0.96246032, 0.95880952]),
 'mean_test_score': array([0.96098413

##### Interpretation
Der mean_test_score verändert sich nicht stark nach Layergröße. Dennoch erhöht sich die Laufzeit pro Layer quadratisch, aber erzielt kaum bessere Testergebnisse

In [9]:
parameter_dist_2 = {
    'mlpclassifier__alpha': [0.0001,0.001,0.01,0.1,1,2,4,8]  
}

clf_alpha = GridSearchCV(MLP, parameter_dist_2,n_jobs=6)
search_alpha=clf_alpha.fit(X_train,y_train)

G:\Users\David\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [10]:
search_alpha.cv_results_

{'mean_fit_time': array([679.12808647, 670.87465792, 635.44919882, 674.28954568,
        529.02818389, 375.73647251, 263.97652178, 181.37300878]),
 'std_fit_time': array([61.70769071, 57.96803236, 47.62810899, 53.8713003 , 50.76709826,
         3.3637014 , 11.10894855, 28.18452625]),
 'mean_score_time': array([0.29141927, 0.26967778, 0.30079608, 0.29580746, 0.31356034,
        0.30079861, 0.29760432, 0.16814475]),
 'std_score_time': array([0.03288767, 0.01854586, 0.00861704, 0.01397227, 0.01562516,
        0.01898472, 0.01916725, 0.0366852 ]),
 'param_mlpclassifier__alpha': masked_array(data=[0.0001, 0.001, 0.01, 0.1, 1, 2, 4, 8],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'mlpclassifier__alpha': 0.0001},
  {'mlpclassifier__alpha': 0.001},
  {'mlpclassifier__alpha': 0.01},
  {'mlpclassifier__alpha': 0.1},
  {'mlpclassifier__alpha': 1},
  {'mlpclassifier__alpha': 2},
  {'mlpclassifier__alp

##### Interpretation
Bei unterschiedlichen Alpha-Werten scheint das Modell nicht besonders sensitiv zu reagieren. Bei höheren alpha-Werten verringert sich die Laufzeit

In [12]:
parameter_dist_3 = {
    'mlpclassifier__learning_rate': ['constant','adaptive','invscaling','power_t']
}

clf_learning_rate = GridSearchCV(MLP, parameter_dist_3,n_jobs=6)
search_learning_rate=clf_learning_rate.fit(X_train,y_train)

G:\Users\David\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
search_learning_rate.cv_results_

{'mean_fit_time': array([650.66032395, 620.88554649, 175.66606541,   2.17857523]),
 'std_fit_time': array([4.81906393e+01, 2.99445951e+01, 1.26026485e+01, 3.72036107e-02]),
 'mean_score_time': array([0.27785811, 0.19467292, 0.27805552, 0.        ]),
 'std_score_time': array([0.01439196, 0.05496089, 0.03636269, 0.        ]),
 'param_mlpclassifier__learning_rate': masked_array(data=['constant', 'adaptive', 'invscaling', 'power_t'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'mlpclassifier__learning_rate': 'constant'},
  {'mlpclassifier__learning_rate': 'adaptive'},
  {'mlpclassifier__learning_rate': 'invscaling'},
  {'mlpclassifier__learning_rate': 'power_t'}],
 'split0_test_score': array([0.95936508, 0.95888889, 0.84428571,        nan]),
 'split1_test_score': array([0.9581746 , 0.9602381 , 0.85484127,        nan]),
 'split2_test_score': array([0.96007937, 0.96214286, 0.8497619 ,        nan]),
 'split3_test_score': ar

##### Interpretation
constant und adaptive unterscheiden sich nicht stark. Beim invscaling reduziert sich der mittlere Testwert start. Dennoch benötigt er im Mean nicht solange Zeit, wie 
das konstante und adpative lernen

##### How hard is it to find the best parameters? How many experiments would you need to find the best parameters?
Mit einem Gridsearch müssten alle Parameter- bzw. viele Parameterkombinationen durchgeprüft werden. Mit einem RandomSearch mit Cross Validierung können oft schneller Ergebnisse 
erzielt werden. Allerdings kann durch den Zufallsaspekt nur das lokale Optimum gefunden werden.